# Crawling Data Twiiter dan Web

## Twitter

### Snscrape

In [ ]:
_= !pip install snscrape

In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [ ]:
def tweet_scrape(tag, count=100):
  list_tweet = []
  for i,tweet in enumerate(sntwitter.TwitterHashtagScraper(tag).get_items()):
    if i > count:
      break
    list_tweet.append({
      'Nama': tweet.user.username, 
      'Content': tweet.content, 
      'Lokasi': tweet.user.location
    })
  return pd.DataFrame(list_tweet) 

In [ ]:
tweet = tweet_scrape('capres2024 AND ganjarpranowo')
tweet.to_csv('ganjarScrape.csv', index=False)

<ipython-input-20-cbc57a5e4014>:8: FutureWarning: content is deprecated, use rawContent instead
  'Content': tweet.content,


### Twint

In [ ]:
_= !pip3 uninstall twint && pip3 install twint && pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [ ]:
_= !pip install nest_asyncio

In [ ]:
import twint
import nest_asyncio
nest_asyncio.apply()

In [ ]:
c = twint.Config()
c.Search = '#capres2024 AND #ganjarpranowo'
c.Lang = 'id'
c.Limit = 1000
# c.Since = "2019-01-01"
# c.Until = "2022-01-01"
c.Pandas = True
twint.run.Search(c)

1627503307253248001 2023-02-20 03:00:00 +0000 <GanjaranApp> "Pejabat yang gemar nonton teater sehat kejiwaannya. Karena menjadi kaidah juga dalam teater sebagai seni mengkritik penguasa" Butet Kertaredjasa  #seniman #seni #teater #butet #butetkartredjasa #ganjarpranowo #ganjarsiap #ganjaranapp #ganjarkita #capres2024  https://t.co/MdW5FVyJUR
1627486484013150208 2023-02-20 01:53:09 +0000 <beritafaktaind> Ganjar Pranowo Cinta Budaya Tanah Air. #ganjarpranowo #budaya #budayaindonesia #pemilu2024 #capres2024 #beritafaktaind  Bu Mega, Indomie, Kakao, Minum, Iphone 14, Rusia  https://t.co/WhqfL1WHml
1627231520258760704 2023-02-19 09:00:01 +0000 <GanjaranApp> “Dengan menonton teater, orang akan terasah perasaannya sehingga menjadikan peka dan tidak pekok.” @ganjarpranowo   #seniman #seni #teater #butet #butetkartredjasa #ganjarpranowo #ganjarsiap #ganjaranapp #ganjarkita #capres2024  https://t.co/oyohxHQb1E
1627201315368583170 2023-02-19 07:00:00 +0000 <GanjaranApp> "Kita sebagai relawan tida

In [ ]:
Tweets_df = twint.storage.panda.Tweets_df

In [ ]:
Tweets_df.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place',
       'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str',
       'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video',
       'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url',
       'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt',
       'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src',
       'trans_dest'],
      dtype='object')

In [ ]:
df = pd.DataFrame({
    'name': Tweets_df['name'],
    'tweet': Tweets_df['tweet'],
    'place': Tweets_df['place'],
})

In [ ]:
df.to_csv('ganjarCapres.csv')

## Web

In [ ]:
_= !pip install bs4

In [ ]:
import requests

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
import pandas as pd

### pta.trunojoyo.ac.id

In [ ]:
class ScrapingPtaTrunojoyo:
  def __init__(self, url):
    self.url = url
  
  def page(self):
    res = requests.get(self.url)
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find('ol', class_='pagination').findChildren(recursive=False)
    return int(contents[-3].text)

  def artikel_page(self, page):
    res = requests.get(f'{self.url}/{page}')
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find_all('li', attrs={'data-cat':'#luxury'})
    return contents

  def get_berita(self):
    pta_list = []
    page = self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in artikels:
        respone = requests.get(artikel.find('a', class_='gray button')['href'])
        soup_data = BeautifulSoup(respone.content, 'html.parser')
        cont = soup_data.find('div', attrs={'id':'content_journal'})
        # ambil data
        nama = cont.findAll('div')[1].div.span.text.split(' : ')[1]
        judul = str.join("", (cont.find('a', class_='title').string).splitlines()).capitalize()
        abstrak = str.join("", (cont.find('p').string).splitlines()).capitalize()
        pta_list.append({
            'Penulis' : nama,
            'Judul': judul,
            'Abstrak': abstrak
        })
    self.pta = pta_list
  
  def result(self):
    return pd.DataFrame(self.pta)
  
  def save(self, namefile='pta'):
    df = pd.DataFrame(self.pta).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [ ]:
scPta = ScrapingPtaTrunojoyo('https://pta.trunojoyo.ac.id/c_search/byprod/10/')

In [ ]:
scPta.get_berita()

In [ ]:
scPta.result()

,Penulis,Judul,Abstrak
0,A.Ubaidillah S.Kom,Perancangan dan implementasi sistem database t...,Sistem informasi akademik (siakad) merupaka...
1,"M. Basith Ardianto,",Aplikasi kontrol dan monitoring jaringan kompu...,Berjalannya koneksi jaringan komputer dengan l...
2,"Akhmad Suyandi, S.Kom",Rancang bangun aplikasi proxy server untukenkr...,Web server adalah sebuah perangkat lunak serve...
3,Heri Supriyanto,Sistem pendukung keputusan optimasi penjadwala...,Penjadwalan kuliah di perguruan tinggi me...
4,Septian Rahman Hakim,Sistem augmented reality animasi benda bergera...,Seiring perkembangan teknologi yang ada diduni...
5,Adi Chandra Laksono,Gerak pekerja pada game real time strategy men...,Gerak pekerja ada pada game yang memiliki genr...
6,NURRACHMAT,Rancang bangun game perawatan sapi karapan men...,"Perkembangan game yang semakin pesat, memberik..."
7,Muhammad Choirur Rozi,Ekstraksi fitur berbasis two dimensional linea...,Sistem pengenalan wajah adalah suatu sistem un...
8,M Khoiril Anwar,Implementasi algoritma prim dan depth first ...,Teknologi mobile game beroperating system open...
9,MALIKUL HAMZAH,Perancangan sistem informasi badan kepegawaian...,Kantor badan kepegawaian kota bangkalan adalah...


In [ ]:
scPta.save()

Berhasil di save


### Kompas

In [38]:
class ScrapingKompas:
  def __init__(self, url):
    self.url = url

  def page(self):
    res_ = requests.get(self.url)
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='paging__link')[:-2]
    return len(contents_)

  def artikel_page(self, page):
    res_ = requests.get(f'{self.url}?page={page}')
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='article__link')
    return contents_

  def get_berita(self):
    berita_list = []
    page = self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in artikels:
        res = requests.get(artikel['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        # ambil data
        judul = soup.find('h1', class_='read__title').string
        isi = isi = ' '.join(soup.find('div', class_='read__content').stripped_strings)
        berita_list.append({
            'Judul': judul,
            'Isi Berita': isi
        })
    self.beritas = berita_list
    return 
  
  def result(self):
    return pd.DataFrame(self.beritas)
  
  def save(self, namefile='beritaKompas'):
    df = pd.DataFrame(self.beritas).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [ ]:
data = ScrapingKompas('https://indeks.kompas.com')

In [39]:
data.get_berita()

In [40]:
data.result()

,Judul,Isi Berita
0,Pengamat Sebut Tak Ada Alasan Mendesak untuk B...,"JAKARTA, KOMPAS.com - Peneliti Lembaga Studi P..."
1,Tujuan TNI-Polri Lakukan Operasi Penegakan Huk...,"MIMIKA , KOMPAS.com - Dalam beberapa hari tera..."
2,4 Tips Mahasiswa Vokasi Cepat Dapat Kerja,"KOMPAS.com - Sejak duduk di bangku kuliah , ma..."
3,Lebih dari 1.200 Pekerja Migran Indonesia Tera...,"LONDON, KOMPAS.com - PT Al Zubara Manpower Ind..."
4,Lirik dan Chord Lagu Love in Store - Fleetwood...,"JAKARTA, KOMPAS.com - “ Love in Store ” adalah..."
5,Mardiono Ajak Sandi Gabung ke PPP untuk Jadi C...,"JAKARTA, KOMPAS.com - Pelaksana Tugas (Plt) Ke..."
6,"Mangsa Hewan Ternak di Ponorogo, Ular Sanca Se...","PONOROGO ,KOMPAS.com - Seeokor ular sanca sepa..."
7,Ketua RT di Lampung Ngamuk Bubarkan Ibadah Ger...,"LAMPUNG , KOMPAS.com - Dugaan persekusi ibadah..."
8,Erick Thohir: FIFA Berharap Indonesia Bisa Gel...,KOMPAS.com - Ketua Umum PSSI Erick Thohir meny...
9,14 Alat Berat Dikerahkan Bersihkan Material Lo...,"KUPANG , KOMPAS.com - Sebanyak 14 unit alat be..."


In [ ]:
data.save()

Berhasil di save


### Detik.com

In [ ]:
class ScrapingDetik:
  def __init__(self, url):
    self.url = url
  
  def page(self):
    res_ = requests.get(self.url)
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.find('div', class_='pagination').findChildren()
    return int(contents_[-2].text)

  def artikel_page(self, page):
    res_ = requests.get(f'{self.url}/{page}')
    soup_ = BeautifulSoup(res_.content, 'html.parser')
    contents_ = soup_.findAll('a', class_='media__link')
    return contents_
  
  def get_berita(self):
    berita_list = []
    page = 5  # self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in range(1, len(artikels), 2):
        res = requests.get(artikels[artikel]['href'])
        soup = BeautifulSoup(res.content, 'html.parser')
        # hapus tag style script
        for d in soup(['style', 'script']):
          d.decompose()
        # ambil data
        judul = ' '.join(soup.find('h1', class_='detail__title').stripped_strings)
        isi = ' '.join(soup.find('div', class_='detail__body-text').stripped_strings)
        berita_list.append({
            'Judul': judul,
            'Isi Berita': isi
        })
    self.beritas = berita_list

  def result(self):
    return pd.DataFrame(self.beritas)
  
  def save(self, namefile='beritaDetik'):
    df = pd.DataFrame(self.beritas).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [ ]:
scrap = ScrapingDetik('https://news.detik.com/indeks')

<a class="media__link" href="https://news.detik.com/pemilu/d-6575185/ketum-ppp-ngaku-siap-hadapi-pemilu-proporsional-terbuka-atau-tertutup" onclick='_pt(this, "newsfeed", "Ketum PPP Ngaku Siap Hadapi Pemilu Proporsional Terbuka atau Tertutup", "artikel 1")'>
<span class="ratiobox ratiobox--4-3 lqd">
<img alt="Ketum PPP Ngaku Siap Hadapi Pemilu Proporsional Terbuka atau Tertutup" class="" src="https://awsimages.detik.net.id/community/media/visual/2023/02/17/ketum-ppp-mardiono_43.jpeg?w=210&amp;q=90" title="Ketum PPP Ngaku Siap Hadapi Pemilu Proporsional Terbuka atau Tertutup"> </img></span>
</a>,

In [ ]:
scrap.get_berita()

[<a class="media__link" href="https://news.detik.com/detiktv/d-6578811/tim-jibom-cek-tkp-ledakan-petasan-maut-blitar-warga-diminta-menjauh" onclick='_pt(this, "newsfeed", "Tim Jibom Cek TKP Ledakan Petasan Maut Blitar, Warga Diminta Menjauh", "artikel 1")'>
<span class="ratiobox ratiobox--4-3 lqd">
<img alt="Tim Jibom Cek TKP Ledakan Petasan Maut Blitar, Warga Diminta Menjauh" class="" src="https://awsimages.detik.net.id/community/media/visual/2023/02/20/ledakan-petasan-di-blitar-3_43.jpeg?w=210&amp;q=90" title="Tim Jibom Cek TKP Ledakan Petasan Maut Blitar, Warga Diminta Menjauh"> </img></span>
</a>, <a class="media__link" href="https://news.detik.com/detiktv/d-6578811/tim-jibom-cek-tkp-ledakan-petasan-maut-blitar-warga-diminta-menjauh" onclick='_pt(this, "newsfeed", "Tim Jibom Cek TKP Ledakan Petasan Maut Blitar, Warga Diminta Menjauh", "artikel 1")'>Tim Jibom Cek TKP Ledakan Petasan Maut Blitar, Warga Diminta Menjauh</a>, <a class="media__link" href="https://news.detik.com/kolom/d-6

In [ ]:
scrap.result()

,Judul,Isi Berita
0,"Tim Jibom Cek TKP Ledakan Petasan Maut Blitar,...",Tim dari Penjinak Bom (Jibom) Polda Jatim bers...
1,Tata Kelola Pengawasan Pemilu,Jakarta - Sebagai lembaga yang diberikan kewen...
2,MA: Hampir Dapat Dipastikan Vonis Mati ke Ferd...,Jakarta - Mahkamah Agung (MA) menyatakan setia...
3,Politisi PKB Minta Kemenkes Gerak Cepat Tangan...,Jakarta - Kementerian Kesehatan (Kemenkes) mel...
4,"Viral Pengedar Narkoba Ngaku Dibekingi Polisi,...",Seorang pengedar narkoba mengaku dibekingi okn...
...,...,...
95,"Sempat Mati, Lampu Merah Kolong Flyover Pasar ...",Jakarta - Lampu merah di kolong flyover Pasar ...
96,"Lubang-lubang Koridor 13 Mulai Ditambal, Pengg...",Jakarta - Lubang-lubang di aspal busway layang...
97,"Heboh 'Gunung Pindah Tutup Jalan' di Kupang, T...",Kupang - Video gundukan tanah menutup jalan di...
98,"Erick & Amali Pimpin PSSI, Syarief Hasan: Tuga...",Jakarta - Menteri BUMN Erick Thohir dan Menter...


In [ ]:
scrap.save()

Berhasil di save
